<a href="https://colab.research.google.com/github/daegeunbae/daegeunbae/blob/main/240706_%EC%88%98%EC%9A%94%EC%98%88%EC%B8%A15_%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81(%EC%B0%B8%EA%B3%A0)_%EC%8B%A0%EA%B2%BD%EB%A7%9D_%ED%99%9C%EC%9A%A9_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 협업 필터링


## 협업 필터링 사용자 기반 유사도

협력적 추천 기법

사용자와 성향이 비슷한 사용자들이 공통적으로 좋아하는 콘텐츠 추천

과거에 이용한 콘텐츠가 비슷하면 유사항 성향이 비슷하다고 봄

용어: 협력적 추천, CF 추천, 협력적 여과, 사회적 여과 등

장점

1.과거 구매 정보만을 가지고 정확도가 높은 추천 가능

2.콘텐츠의 내용정보, 사용자의 인구통계학적 정보를 전혀 필요로 하지 않음

예시: 도서추천, 영화추천, 음악추천

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
ratings=pd.read_table("/content/drive/MyDrive/02.coding/01_3.수요예측/u.data", header=None)
ratings.columns=["user","item","rating","timestamp"]
ratings.head()

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user       100000 non-null  int64
 1   item       100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


In [4]:
#최대값과 최소값 확인
ratings["user"].max(),ratings["item"].max(), ratings["user"].min(),ratings["item"].min()

(943, 1682, 1, 1)

In [5]:
# user와 item 부분의 값을 0부터 시작하도록 셋팅
ratings["user"]=ratings["user"]-1
ratings["item"]=ratings["item"]-1
ratings["user"].max(),ratings["item"].max(), ratings["user"].min(),ratings["item"].min()

(942, 1681, 0, 0)

In [6]:
#조정된 값의 최대값과 최소값 확인
n_users=ratings["user"].max()+1
n_items=ratings["item"].max()+1
shape=(n_users, n_items)
#인접 행렬의 크기를 나타내는 튜플
shape

(943, 1682)

### 사용자, item 간의 인접 행렬 이용=> 0,1 로 구성

In [7]:
# 사용자, item 간의 인접 행렬
#사용자-항목 간의 관계를 이진 형식으로 표현합니다. 특정 사용자가 특정 항목을 평가했는지 여부를 0과 1로 나타내어,
#추천 시스템이나 기타 분석 작업에서 활용할 수 있는 기초 데이터를 제공
import numpy as np
adj_matrix=np.zeros(shape, dtype=np.int8)
for user, item, rating, timestamp in ratings.values:
  adj_matrix[user, item]=1
adj_matrix

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int8)

#### 유사도 함수 (내적)

주어진 사용자의 벡터를 다른 사용자들의 벡터와 비교하여 가장 유사한 사용자를 찾는 과정을 수행

여기서 유사도는 내적(dot product)을 통해 계산

In [8]:
#my_id는 비교 대상이 되는 사용자의 ID 여기서는 0번 사용자
#my_vector는 adj_matrix의 첫 번째 행으로, 0번 사용자의 벡터. 이 벡터는 해당 사용자가 평가한 항목을 나타냄
my_id, my_vector=0 , adj_matrix[0]
#best_match는 가장 높은 유사도 값을 저장하기 위한 변수로, 초기값은 -1
#best_match_id는 가장 유사한 사용자의 ID를 저장하기 위한 변수
#best_match_vector는 가장 유사한 사용자의 벡터를 저장하기 위한 변수
best_match, best_match_id, best_match_vector=-1,-1, []

#다른 사용자와의 유사도 계산
for user_id, vector in enumerate(adj_matrix):
  if user_id==my_id:
    continue
  similarity=(my_vector* vector).sum()
  if similarity > best_match:
    best_match=similarity
    best_match_id=user_id
    best_match_vector=vector

#가장 높은 유사도 값(best_match)과 가장 유사한 사용자의 ID(best_match_id)를 출력
best_match, best_match_id

(183, 275)

In [9]:
#추천 리스트 초기화
recommend_list=[]
#벡터 순회 및 조건 체크
for id, log in enumerate(zip(my_vector,best_match_vector)):
  log1, log2=log
  if log1==0 and log2>0:
    recommend_list.append(id)
#my_vector에서 평가하지 않았으나, best_match_vector에서 평가된 항목의 인덱스를 포함
recommend_list

[272,
 273,
 275,
 280,
 281,
 283,
 287,
 288,
 289,
 290,
 292,
 293,
 297,
 299,
 300,
 301,
 302,
 306,
 312,
 314,
 315,
 316,
 317,
 321,
 322,
 323,
 324,
 327,
 330,
 331,
 332,
 333,
 339,
 342,
 345,
 346,
 353,
 354,
 355,
 356,
 357,
 363,
 364,
 365,
 366,
 372,
 374,
 378,
 379,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 390,
 391,
 392,
 394,
 395,
 396,
 398,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 412,
 414,
 416,
 417,
 418,
 419,
 420,
 422,
 424,
 425,
 426,
 427,
 428,
 430,
 431,
 432,
 435,
 442,
 446,
 447,
 448,
 449,
 450,
 451,
 452,
 454,
 455,
 457,
 460,
 461,
 462,
 468,
 469,
 470,
 471,
 472,
 473,
 474,
 478,
 495,
 500,
 507,
 517,
 522,
 525,
 530,
 539,
 540,
 543,
 545,
 546,
 548,
 549,
 550,
 551,
 553,
 557,
 558,
 560,
 561,
 562,
 563,
 565,
 566,
 567,
 568,
 570,
 571,
 574,
 575,
 576,
 577,
 580,
 581,
 582,
 585,
 587,
 589,
 590,
 594,
 596,
 602,
 623,
 626,
 627,
 630,
 633,
 635,
 639,
 646,
 648,
 651,
 652

#### 유사도 함수 : cossin 이용

In [10]:
#필요 라이브러리를 임포트하고 초기 변수를 설정
from sklearn.metrics.pairwise import cosine_similarity
#my_vector와 다른 모든 사용자의 벡터 간의 코사인 유사도를 계산
my_id, my_vector=0 , adj_matrix[0]
#가장 높은 유사도를 가진 사용자를 찾고, 그 사용자의 ID와 벡터를 저장
best_match, best_match_id, best_match_vector=-1,-1, []
#최종적으로 가장 유사한 사용자의 유사도 값과 ID를 출력
for user_id, vector in enumerate(adj_matrix):
  if user_id==my_id:
    continue
  length=len(my_vector)
  similarity=cosine_similarity(my_vector.reshape(1,length), vector.reshape(1,length)).reshape(-1)
  if similarity > best_match:
    best_match=similarity
    best_match_id=user_id
    best_match_vector=vector
best_match, best_match_id

(array([0.52785862]), 915)

In [11]:
cosine_similarity(my_vector.reshape(1,length), adj_matrix[275].reshape(1,length)).reshape(-1)

array([0.48753046])

###  rating matrix 이용

In [12]:
# 사용자, item 간의 인접 행렬
import numpy as np
rating_matrix=np.zeros(shape, dtype=np.int8)
for user, item, rating, timestamp in ratings.values:
  rating_matrix[user, item]=rating
rating_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]], dtype=int8)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
my_id, my_vector=0 , rating_matrix[0]
best_match, best_match_id, best_match_vector=-1,-1, []
for user_id, vector in enumerate(rating_matrix):
  if user_id==my_id:
    continue
  length=len(my_vector)
  similarity=cosine_similarity(my_vector.reshape(1,length), vector.reshape(1,length)).reshape(-1)
  if similarity > best_match:
    best_match=similarity
    best_match_id=user_id
    best_match_vector=vector
best_match, best_match_id

(array([0.56906573]), 915)

In [14]:
cos_similarity=cosine_similarity(rating_matrix, rating_matrix)
cos_similarity.shape

(943, 943)

In [15]:
np.argsort(cos_similarity[0])[-10:]

array([302, 428, 737, 456, 434,  91, 267, 863, 915,   0])

#### 전체 user의 코싸인 유사도 행렬

In [16]:
def cos_similarity(A, B):
  return (A*B).sum()/(np.linalg.norm(A)*np.linalg.norm(B) )
length=rating_matrix.shape[0]
cos_matrix=np.zeros((length,length))
for i in range(length):
  for j in range(length):
    cos_matrix[i,j]=cos_similarity(rating_matrix[i], rating_matrix[j])

In [17]:
np.sort(cos_matrix[0])[-10:]

array([0.52571773, 0.52594993, 0.52703107, 0.53847598, 0.53866453,
       0.54053356, 0.54207705, 0.54754826, 0.56906573, 1.        ])

In [18]:
np.argsort(cos_matrix[0])[-10:]

array([302, 428, 737, 456, 434,  91, 267, 863, 915,   0])

In [19]:
best_match_vector=rating_matrix[863]
best_match_vector
recommend_list=[]
for id, log in enumerate(zip(my_vector,best_match_vector)):
  log1, log2=log
  if log1==0 and log2>0:
    recommend_list.append(id)
recommend_list

[272,
 274,
 275,
 281,
 282,
 285,
 287,
 289,
 293,
 316,
 317,
 327,
 332,
 342,
 348,
 355,
 356,
 366,
 372,
 379,
 381,
 385,
 390,
 392,
 393,
 398,
 400,
 401,
 402,
 403,
 404,
 407,
 417,
 418,
 421,
 422,
 431,
 432,
 442,
 450,
 464,
 465,
 469,
 470,
 471,
 472,
 473,
 475,
 482,
 495,
 500,
 508,
 510,
 522,
 525,
 530,
 540,
 541,
 545,
 548,
 549,
 558,
 560,
 561,
 562,
 565,
 567,
 568,
 576,
 577,
 587,
 590,
 595,
 596,
 602,
 608,
 618,
 622,
 624,
 627,
 628,
 641,
 650,
 654,
 657,
 659,
 662,
 664,
 671,
 672,
 677,
 683,
 684,
 691,
 692,
 707,
 709,
 714,
 715,
 716,
 719,
 721,
 728,
 731,
 733,
 734,
 735,
 741,
 746,
 754,
 767,
 769,
 774,
 779,
 780,
 788,
 793,
 796,
 799,
 800,
 804,
 891,
 929,
 938,
 950,
 965,
 968,
 971,
 992,
 1015,
 1032,
 1043,
 1046,
 1100,
 1108,
 1111,
 1118,
 1134,
 1139,
 1207,
 1209,
 1216,
 1227,
 1247,
 1283,
 1302,
 1411,
 1424,
 1445,
 1530]

#### 전체 user의 corr 유사도 행렬

In [20]:
def corr_similarity(A, B):
  return ((A-A.mean())*(B-B.mean())).mean()/(A.std()*B.std())
corr_similarity(rating_matrix[0], rating_matrix[1])

0.1063219297355773

In [21]:
length=rating_matrix.shape[0]
corr_matrix=np.zeros((length,length))
for i in range(length):
  for j in range(length):
    corr_matrix[i,j]=corr_similarity(rating_matrix[i], rating_matrix[j])

In [22]:
np.argsort(corr_matrix[0])[-10:]

array([434, 513,  91, 822, 267, 456, 863, 737, 915,   0])

In [23]:
set([302, 428, 737, 456, 434,  91, 267, 863, 915,   0]).intersection(set([434, 513,  91, 822, 267, 456, 863, 737, 915,   0]))

{0, 91, 267, 434, 456, 737, 863, 915}

In [24]:
best_match_vector=rating_matrix[915]
best_match_vector
recommend_list=[]
for id, log in enumerate(zip(my_vector,best_match_vector)):
  log1, log2=log
  if log1==0 and log2>4:
    recommend_list.append(id)
recommend_list

[420, 424, 482, 510, 511, 918, 1008]

# 아이템 기반 협력 필터링

## 데이터 가져오기

In [25]:
import pandas as pd
ratings=pd.read_table("/content/drive/MyDrive/02.coding/01_3.수요예측/u.data", header=None)
ratings.columns=["user","item","rating","timestamp"]
ratings.head()

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## user-item rating_matrix

In [26]:
# user와 item 부분의 값을 0부터 시작하도록 셋팅
ratings["user"]=ratings["user"]-1
ratings["item"]=ratings["item"]-1
ratings["user"].max(),ratings["item"].max(), ratings["user"].min(),ratings["item"].min()

(942, 1681, 0, 0)

In [27]:
# 사용자, item 간의 인접 행렬
shape=(ratings["user"].max()+1, ratings["item"].max()+1)
import numpy as np
rating_matrix=np.zeros(shape, dtype=np.int8)
for user, item, rating, timestamp in ratings.values:
  rating_matrix[user, item]=rating
#rating_matrix

## 아이템 기반 유사도

In [28]:
def cos_similarity(A, B):
  return (A*B).sum()/(np.linalg.norm(A)*np.linalg.norm(B) )
# item 유사도
length=rating_matrix.shape[1]
cos_matrix=np.zeros((length,length))
for i in range(length):
  for j in range(length):
    cos_matrix[i,j]=cos_similarity(rating_matrix.T[i], rating_matrix.T[j]) #rating_matrix.T : transpose matrix 이용
cos_matrix.shape

(1682, 1682)

In [29]:
np.where(np.array([1,0,0,1,0])==0)

(array([1, 2, 4]),)

In [30]:
def generate_recommendations(user_id, user_item_matrix, similarity_matrix, n_recommendations=5):
  user_ratings=user_item_matrix[user_id]
  user_unrated_items = np.where(user_ratings==0)[0]
  print(user_unrated_items)
  # 예상 평점 계산
  predictions = {}
  #각 아이템이 대한  예측 점수
  for item in user_unrated_items:
    item_similarity = similarity_matrix[item]
    user_ratings_weighted = user_ratings*item_similarity
    prediction = (user_ratings_weighted.sum()) / (item_similarity.sum())
    predictions[item] = prediction

  #최고의 예상 평점을 가진 아이템 추천
  recommended_items = sorted(predictions.items(), key=lambda x: x[1],reverse=True)[:n_recommendations]
  if len(recommended_items)<5:
    return recommended_items
  return recommended_items[:n_recommendations]

In [31]:
recommended_movies = generate_recommendations(0, rating_matrix, cos_matrix, 3)
recommended_movies

[ 272  273  274 ... 1679 1680 1681]


[(1617, 1.5287323927638843),
 (1681, 1.5037109034147254),
 (1129, 1.4887880562603861)]

# 회귀 분석을 위한 신경망 모델

## 데이터 셋팅

In [32]:
import pandas as pd
concrete=pd.read_csv('/content/drive/MyDrive/02.coding/01_3.수요예측/concrete_na.csv')
# 중복치 제거
concrete=concrete.drop_duplicates()
# 결측치 제거
concrete=concrete.dropna()
# input, target split
X=concrete.drop("strength", axis=1)
y=concrete["strength"]
# train,test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values.astype("float"), y.values.astype("float"), test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((796, 8), (199, 8), (796,), (199,))

In [33]:
X_train.dtype

dtype('float64')

In [34]:
y_train.dtype

dtype('float64')

## 모델 설계와 훈련

이 모델은 입력 데이터로 8개의 특성을 받고, 중간에 256개의 뉴런을 가지는 층을 거쳐, 최종적으로 1개의 출력 값을 예측하는 간단한 인공신경망

In [35]:
#입력 데이터의 형태를 지정하는 입력층을 추가
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
# 모델 구조
model=Sequential()
model.add(Input((8,)))
#첫 번째 밀집층을 추가하여 256개의 뉴런을 정의
model.add(Dense(256))
#model.add(Dense(128))
#출력층을 추가하여 단일 값을 출력
model.add(Dense(1))
#model.summary()를 통해 모델의 구조를 요약하여 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2304      
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2561 (10.00 KB)
Trainable params: 2561 (10.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
#https://www.tensorflow.org/api_docs/python/tf/keras/metrics/R2Score
from tensorflow.keras.metrics import R2Score
r2score=R2Score()

In [37]:
from tensorflow.keras.optimizers import Adam
# 컴파일 mae는 큰 에러에 더 큰 페널티를 줌
model.compile(loss="mse", optimizer=Adam(learning_rate=0.0005), metrics=["mae",r2score])#metrics: 검증 지표
history=model.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test))

Epoch 1/300
25/25 [==============================] - 1s 14ms/step - loss: 8462.8057 - mae: 70.6462 - r2_score: -30.9773 - val_loss: 1385.0972 - val_mae: 31.0666 - val_r2_score: -4.1579
Epoch 2/300
25/25 [==============================] - 0s 4ms/step - loss: 1210.9255 - mae: 28.1194 - r2_score: -3.5756 - val_loss: 605.6551 - val_mae: 20.0953 - val_r2_score: -1.2554
Epoch 3/300
25/25 [==============================] - 0s 3ms/step - loss: 570.0035 - mae: 19.1687 - r2_score: -1.1538 - val_loss: 461.3397 - val_mae: 17.6131 - val_r2_score: -0.7180
Epoch 4/300
25/25 [==============================] - 0s 3ms/step - loss: 412.2557 - mae: 16.2058 - r2_score: -0.5577 - val_loss: 374.2036 - val_mae: 16.0702 - val_r2_score: -0.3935
Epoch 5/300
25/25 [==============================] - 0s 4ms/step - loss: 333.2719 - mae: 14.7055 - r2_score: -0.2593 - val_loss: 293.3773 - val_mae: 14.2386 - val_r2_score: -0.0925
Epoch 6/300
25/25 [==============================] - 0s 3ms/step - loss: 256.7812 - mae: 1

# 신경망을 이용한 분류 문제

## 분류를 위한 데이터 셋팅

In [38]:
# strength 를 4개의 균등한 구간으로 분리하기 => pd.cut()  ["w", "n", "s", "st"]
import pandas as pd
concrete=pd.read_csv('/content/drive/MyDrive/02.coding/01_3.수요예측/concrete_na.csv')
# 중복치 제거
concrete=concrete.drop_duplicates()
# 결측치 제거
concrete=concrete.dropna()
# strength 를 4개의 균등한 구간으로 분리하기 => pd.cut()  [0,1,2,3]
concrete["label"]=pd.cut(concrete["strength"], bins=4, labels=[0,1,2,3])

concrete.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength,label
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28.0,29.89,1
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14.0,23.51,1
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28.0,29.22,1
3,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28.0,45.85,2
4,154.8,183.4,0.0,193.3,9.1,1047.4,696.7,28.0,18.29,0


In [39]:
# input target 분리
X=concrete.drop(["strength", "label" ], axis=1)
y=concrete["label"]
# 타겟 원-핫 인코딩
from tensorflow.keras.utils import to_categorical
y_encoded=to_categorical(y.values, 4)

# train test 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values.astype("float"), y_encoded, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((796, 8), (199, 8), (796, 4), (199, 4))

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
# 모델 구조
# 랜덤 씨드 설정
import tensorflow as tf
tf.random.set_seed(123)
np.random.seed(123)


model=Sequential()
model.add(Input((8,)))
model.add(Dense(128, activation="relu"))
model.add(Dense(4, activation="relu"))
model.compile(loss="categorical_crossentropy", optimizer=Adam(0.002), metrics=["accuracy"])#metrics: 검증 지표
history=model.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test))

Epoch 1/300
25/25 [==============================] - 1s 15ms/step - loss: 11.3332 - accuracy: 0.2487 - val_loss: 12.2798 - val_accuracy: 0.1859
Epoch 2/300
25/25 [==============================] - 0s 3ms/step - loss: 9.8624 - accuracy: 0.2839 - val_loss: 4.6774 - val_accuracy: 0.5075
Epoch 3/300
25/25 [==============================] - 0s 3ms/step - loss: 4.3066 - accuracy: 0.4636 - val_loss: 4.6388 - val_accuracy: 0.5075
Epoch 4/300
25/25 [==============================] - 0s 4ms/step - loss: 4.2957 - accuracy: 0.4636 - val_loss: 4.6173 - val_accuracy: 0.5075
Epoch 5/300
25/25 [==============================] - 0s 4ms/step - loss: 4.2810 - accuracy: 0.4636 - val_loss: 4.6073 - val_accuracy: 0.5075
Epoch 6/300
25/25 [==============================] - 0s 3ms/step - loss: 4.2672 - accuracy: 0.4636 - val_loss: 4.5796 - val_accuracy: 0.5075
Epoch 7/300
25/25 [==============================] - 0s 4ms/step - loss: 4.2444 - accuracy: 0.4673 - val_loss: 4.5758 - val_accuracy: 0.5226
Epoch 8/30

이 모든 과정은 머신러닝 모델을 구축하고 학습시키기 위한 전형적인 단계들.

각 단계는 모델이 데이터를 통해 패턴을 학습하고, 이를 통해 예측 성능을 높이며, 과적합을 방지하고, 결과를 재현 가능하게 만드는 데 목적이 있음.

이러한 과정은 최종적으로 실전 데이터에 적용하여 유용한 예측 모델을 만드는 데 기여함